<a href="https://colab.research.google.com/github/feliciahf/NLP-Project/blob/main/amazon/colab_FH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

Check whether stemming & lemmatization make difference

Do we need bigram/trigram frequencies (within titles)?

Do we need dataframe including cleaned data & genres? (which cleaned data?) -> create dictionary of lists maybe
https://www.geeksforgeeks.org/python-ways-to-create-a-dictionary-of-lists/

How do we incorporate POS-tagging & NER in classifier?

# Importing & Cleaning data

This section imports the data into a pandas dataframe and goes through the following preprocessing steps:

-Case collapsing 
-Remove punctuation 
-Tokenization 
-N-Grams: bigrams and trigrams 
-Stemming -> check whether this makes a difference 
-Lemmatization -> check whether this makes a difference 
-Part-of-speech (POS) tagging 
-Named entity recognition (NER) 

## Import Data
Imports data as pandas dataframe
Create list of title and genre columns from original data
Create list including all 32 genres

In [2]:
from google.colab import files
uploaded = files.upload()

Saving book32listing.csv to book32listing.csv


In [3]:
import pandas as pd
df = pd.read_csv("book32listing.csv", encoding='latin1', header=None)
df1 = df[[3,6]] # only columns with titles and genres
df1.columns = ['title', 'genre']
print(df1)

                                                    title      genre
0                         Mom's Family Wall Calendar 2016  Calendars
1                         Doug the Pug 2016 Wall Calendar  Calendars
2       Moleskine 2016 Weekly Notebook, 12M, Large, Bl...  Calendars
3                 365 Cats Color Page-A-Day Calendar 2016  Calendars
4                    Sierra Club Engagement Calendar 2016  Calendars
...                                                   ...        ...
207567  ADC the Map People Washington D.C.: Street Map...     Travel
207568  Washington, D.C., Then and Now: 69 Sites Photo...     Travel
207569  The Unofficial Guide to Washington, D.C. (Unof...     Travel
207570      Washington, D.C. For Dummies (Dummies Travel)     Travel
207571  Fodor's Where to Weekend Around Boston, 1st Ed...     Travel

[207572 rows x 2 columns]


In [4]:
titles = df1['title'] # list of all titles
titles1 = titles.values.tolist() # change to list of strings
print(titles1[0:6]) # test whether it worked

["Mom's Family Wall Calendar 2016", 'Doug the Pug 2016 Wall Calendar', 'Moleskine 2016 Weekly Notebook, 12M, Large, Black, Soft Cover (5 x 8.25)', '365 Cats Color Page-A-Day Calendar 2016', 'Sierra Club Engagement Calendar 2016', 'Sierra Club Wilderness Calendar 2016']


In [17]:
genres = df1['genre']
genres = genres.values.tolist()
genres = pd.DataFrame(genres)

In [18]:
df1.genre.unique() # list of all possible genres

array(['Calendars', 'Comics & Graphic Novels', 'Test Preparation',
       'Mystery, Thriller & Suspense', 'Science Fiction & Fantasy',
       'Romance', 'Humor & Entertainment', 'Literature & Fiction',
       'Gay & Lesbian', 'Engineering & Transportation',
       'Cookbooks, Food & Wine', 'Crafts, Hobbies & Home',
       'Arts & Photography', 'Education & Teaching',
       'Parenting & Relationships', 'Self-Help', 'Computers & Technology',
       'Medical Books', 'Science & Math', 'Health, Fitness & Dieting',
       'Business & Money', 'Law', 'Biographies & Memoirs', 'History',
       'Politics & Social Sciences', 'Reference',
       'Christian Books & Bibles', 'Religion & Spirituality',
       'Sports & Outdoors', 'Teen & Young Adult', "Children's Books",
       'Travel'], dtype=object)

## Case Collapsing
Change all uppercase to lowercase letters

In [6]:
case_collap = map(lambda x:x.lower(), titles1)
case_collap_list = list(case_collap)
print(case_collap_list[0:6])

["mom's family wall calendar 2016", 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook, 12m, large, black, soft cover (5 x 8.25)', '365 cats color page-a-day calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Remove Punctuation
Remove punctuation by creating translation table
Punctuation to be removed is given in string: string.punctuation

In [7]:
import string
trans = str.maketrans('', '', string.punctuation)
rem_punct = [s.translate(trans) for s in case_collap_list]
print(rem_punct[0:6])

['moms family wall calendar 2016', 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook 12m large black soft cover 5 x 825', '365 cats color pageaday calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Tokenization
Split all titles into words
Output: list of lists of strings

In [10]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
tokenized_titles = [word_tokenize(i) for i in rem_punct]
print(tokenized_titles[:10])

[['moms', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cats', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016'], ['thomas', 'kinkade', 'the', 'disney', 'dreams', 'collection', '2016', 'wall', 'calendar'], ['ansel', 'adams', '2016', 'wall', 'calendar'], ['dilbert', '2016', 'daytoday', 'calendar'], ['mary', 'engelbreit', '2016', 'deluxe', 'wall', 'calendar', 'never', 'give', 'up']]


## N-Grams: Bigrams and Trigrams
Create bigrams + trigrams
(could be done in one go e.g. n=2 for bigrams, n=3 for trigrams etc.

In [12]:
# bigrams
token_bigram = []
for title in tokenized_titles:
    title_bigram = []
    for w in range(len(title) - 1):
        title_bigram.append([title[w], title[w + 1]])
    token_bigram.append(title_bigram)
print(token_bigram[:6]) # test whether working

[[['moms', 'family'], ['family', 'wall'], ['wall', 'calendar'], ['calendar', '2016']], [['doug', 'the'], ['the', 'pug'], ['pug', '2016'], ['2016', 'wall'], ['wall', 'calendar']], [['moleskine', '2016'], ['2016', 'weekly'], ['weekly', 'notebook'], ['notebook', '12m'], ['12m', 'large'], ['large', 'black'], ['black', 'soft'], ['soft', 'cover'], ['cover', '5'], ['5', 'x'], ['x', '825']], [['365', 'cats'], ['cats', 'color'], ['color', 'pageaday'], ['pageaday', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'engagement'], ['engagement', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'wilderness'], ['wilderness', 'calendar'], ['calendar', '2016']]]


In [13]:
# trigrams
token_trigram = []
for title in tokenized_titles:
    title_trigram = []
    for w in range(len(title) - 2):
        title_trigram.append([title[w], title[w + 1], title[w + 2]])
    token_trigram.append(title_trigram)
print(token_trigram[:6]) # test whether working

[[['moms', 'family', 'wall'], ['family', 'wall', 'calendar'], ['wall', 'calendar', '2016']], [['doug', 'the', 'pug'], ['the', 'pug', '2016'], ['pug', '2016', 'wall'], ['2016', 'wall', 'calendar']], [['moleskine', '2016', 'weekly'], ['2016', 'weekly', 'notebook'], ['weekly', 'notebook', '12m'], ['notebook', '12m', 'large'], ['12m', 'large', 'black'], ['large', 'black', 'soft'], ['black', 'soft', 'cover'], ['soft', 'cover', '5'], ['cover', '5', 'x'], ['5', 'x', '825']], [['365', 'cats', 'color'], ['cats', 'color', 'pageaday'], ['color', 'pageaday', 'calendar'], ['pageaday', 'calendar', '2016']], [['sierra', 'club', 'engagement'], ['club', 'engagement', 'calendar'], ['engagement', 'calendar', '2016']], [['sierra', 'club', 'wilderness'], ['club', 'wilderness', 'calendar'], ['wilderness', 'calendar', '2016']]]


## Stemming
Test this out to see whether it makes a difference in final classifier

PorterStemmer (one algorithm for stemming; less aggressive than LancasterStemming)

Create empty list to contain lists of stems in each title
Create empty list for stems of title
Add each stemmed word in title to second list
Append list of stemmed words to first list

In [14]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()
stems = []   
for title in tokenized_titles:
    stems_title = []
    for word in title:
        stems_title.append(porter.stem(word))
    stems.append(stems_title)
    
print(stems[0:6]) # test whether it works

[['mom', 'famili', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskin', '2016', 'weekli', 'notebook', '12m', 'larg', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engag', 'calendar', '2016'], ['sierra', 'club', 'wilder', 'calendar', '2016']]


## Lemmatization
Same principle as with stemming
Test this out to see whether it makes a difference in final classifier

In [15]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemmas = []
for title in tokenized_titles:
    lemmas_title = []
    for word in title:
        lemmas_title.append(lemmatizer.lemmatize(word))
    lemmas.append(lemmas_title)
    
print(lemmas[0:6]) # test whether it works

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[['mom', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016']]


## Part-of-Speech (POS) Tagging
Create list of lists with tokens and their corresponding part-of-speech tag in each title

In [16]:
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag

postag = []
for title in tokenized_titles:
    postag.append(nltk.pos_tag(title))
    
print(postag[0:6]) # testing whether postag worked

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[[('moms', 'NNS'), ('family', 'NN'), ('wall', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('doug', 'VB'), ('the', 'DT'), ('pug', 'NN'), ('2016', 'CD'), ('wall', 'NN'), ('calendar', 'NN')], [('moleskine', 'NN'), ('2016', 'CD'), ('weekly', 'JJ'), ('notebook', 'NN'), ('12m', 'CD'), ('large', 'JJ'), ('black', 'JJ'), ('soft', 'JJ'), ('cover', 'NN'), ('5', 'CD'), ('x', 'JJ'), ('825', 'CD')], [('365', 'CD'), ('cats', 'NNS'), ('color', 'VBP'), ('pageaday', 'IN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('engagement', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('wilderness', 'NN'), ('calendar', 'NN'), ('2016', 'CD')]]


## Named Entity Recognition (NER)
Create list of lists with words, correspondent POS and named entity tags for each title
This uses postags created in previous step

In [19]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import ne_chunk
from nltk.chunk import tree2conlltags

ner = []
for title in titles1:
    ner.append(tree2conlltags(ne_chunk(pos_tag(word_tokenize(title)))))
    
print(ner[0:6]) # test whether NER worked

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[[('Mom', 'NNP', 'B-PERSON'), ("'s", 'POS', 'O'), ('Family', 'NNP', 'B-PERSON'), ('Wall', 'NNP', 'I-PERSON'), ('Calendar', 'NNP', 'I-PERSON'), ('2016', 'CD', 'O')], [('Doug', 'NNP', 'O'), ('the', 'DT', 'O'), ('Pug', 'NNP', 'O'), ('2016', 'CD', 'O'), ('Wall', 'NNP', 'B-FACILITY'), ('Calendar', 'NNP', 'I-FACILITY')], [('Moleskine', 'NN', 'O'), ('2016', 'CD', 'O'), ('Weekly', 'NNP', 'O'), ('Notebook', 'NNP', 'O'), (',', ',', 'O'), ('12M', 'CD', 'O'), (',', ',', 'O'), ('Large', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Black', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Soft', 'NNP', 'B-PERSON'), ('Cover', 'NNP', 'I-PERSON'), ('(', '(', 'O'), ('5', 'CD', 'O'), ('x', 'RB', 'O'), ('8.25', 'CD', 'O'), (')', ')', 'O')], [('365', 'CD', 'O'), ('Cats', 'NNPS', '

# Classifiers
Check whether stemming/lemmatization make difference in final classifiers
-> does it improve/worsen classifier?


-Naive Bayes (bag-of-words)

-BERT (language model)

https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794

In [20]:
# create dataframe containing tokenized titles and genres
tok_title = pd.DataFrame({0: tokenized_titles})
data_in = [tok_title[0], df1["genre"]]
headers = ["titles", "genres"]

data = pd.concat(data_in, axis=1, keys=headers)
print(data[:5])

                                              titles     genres
0               [moms, family, wall, calendar, 2016]  Calendars
1             [doug, the, pug, 2016, wall, calendar]  Calendars
2  [moleskine, 2016, weekly, notebook, 12m, large...  Calendars
3       [365, cats, color, pageaday, calendar, 2016]  Calendars
4         [sierra, club, engagement, calendar, 2016]  Calendars


In [21]:
# split data into train and test
import numpy as np

test_pct=0.2 # split into 80/20%

# create mask
mask = np.random.choice([0, 1], p=[1 - test_pct, test_pct], size=data.shape[0])

# apply mask
data["mask"] = mask
test = data[data["mask"] == 1]
train = data[data["mask"] == 0]

# removing column
test = test.drop("mask", axis="columns").reset_index()
train = train.drop("mask", axis="columns").reset_index()

# remove original indexing data (otherwise we have double indexing)
test = test.drop("index", axis="columns")
train = train.drop("index", axis="columns")

## Naive Bayes classifier

split into training and test data -> 80% / 20%
use n-grams ?

build vocabulary -> need token frequencies across all titles (create sublist with tokenized_titles)
vocabulary for each genre

In [22]:
from collections import Counter

In [23]:
# token frequencies within each title
unigram_count = []
for title in tokenized_titles:
    uni_title = Counter()
    for i in title:
        uni_title[i] += 1
    unigram_count.append(uni_title)

print(unigram_count[:5]) # test

[Counter({'moms': 1, 'family': 1, 'wall': 1, 'calendar': 1, '2016': 1}), Counter({'doug': 1, 'the': 1, 'pug': 1, '2016': 1, 'wall': 1, 'calendar': 1}), Counter({'moleskine': 1, '2016': 1, 'weekly': 1, 'notebook': 1, '12m': 1, 'large': 1, 'black': 1, 'soft': 1, 'cover': 1, '5': 1, 'x': 1, '825': 1}), Counter({'365': 1, 'cats': 1, 'color': 1, 'pageaday': 1, 'calendar': 1, '2016': 1}), Counter({'sierra': 1, 'club': 1, 'engagement': 1, 'calendar': 1, '2016': 1})]


In [24]:
# token frequencies in total vocab
tok_freq = Counter()
for title in train['titles']:
    for i in title:
        tok_freq[i] += 1
        
print(tok_freq)

Counter({'the': 92278, 'and': 54432, 'of': 51033, 'a': 31166, 'to': 27916, 'for': 21219, 'in': 21205, 'guide': 16079, 'edition': 12401, 'book': 8142, 'with': 7877, 'your': 7329, 'from': 6472, 'series': 6328, 'on': 5904, 'how': 5878, 'an': 5153, 'new': 4792, 'life': 4646, 'world': 4019, 'volume': 3987, 'history': 3935, 'you': 3806, '1': 3279, 'recipes': 3185, 'by': 3173, 'american': 2995, 'travel': 2935, 'complete': 2782, 'law': 2679, 'story': 2671, '2': 2669, 'art': 2599, 'handbook': 2501, 'my': 2485, 'health': 2448, 'books': 2389, 'science': 2293, 'calendar': 2182, 'at': 2173, 'novel': 2043, 'practice': 2036, 'best': 1999, 'i': 1971, 'guides': 1945, 'map': 1920, 'cookbook': 1918, 'that': 1868, 'vol': 1836, 'about': 1814, 'love': 1797, '2016': 1752, 'what': 1744, 'great': 1692, 'stories': 1684, 'more': 1628, 'business': 1626, 'war': 1624, 'is': 1609, 'it': 1603, 'introduction': 1563, 'one': 1546, 'second': 1543, 'america': 1536, 'modern': 1531, 'manual': 1530, 'other': 1515, 'managemen

In [25]:
# group data by genre
# could probably be done MUCH nicer, but couldn't get for-loop to work...

grouped = train.groupby(train.genres)

Calendars = grouped.get_group("Calendars")
Comics = grouped.get_group("Comics & Graphic Novels")
Test = grouped.get_group("Test Preparation")
Mystery = grouped.get_group("Mystery, Thriller & Suspense")
SciFi = grouped.get_group("Science Fiction & Fantasy")
Romance = grouped.get_group("Romance")
Humor = grouped.get_group("Humor & Entertainment")
Literature = grouped.get_group("Literature & Fiction")
LGBTQ = grouped.get_group("Gay & Lesbian")
Engineering = grouped.get_group("Engineering & Transportation")
Food = grouped.get_group("Cookbooks, Food & Wine")
Crafts = grouped.get_group("Crafts, Hobbies & Home")
Arts = grouped.get_group("Arts & Photography")
Education = grouped.get_group("Education & Teaching")
Parenting = grouped.get_group("Parenting & Relationships")
SelfHelp = grouped.get_group("Self-Help")
Computers = grouped.get_group("Computers & Technology")
Medical = grouped.get_group("Medical Books")
Science = grouped.get_group("Science & Math")
Health = grouped.get_group("Health, Fitness & Dieting")
Business = grouped.get_group("Business & Money")
Law = grouped.get_group("Law")
Biographies = grouped.get_group("Biographies & Memoirs")
History = grouped.get_group("History")
Politics = grouped.get_group("Politics & Social Sciences")
Reference = grouped.get_group("Reference")
Bibles = grouped.get_group("Christian Books & Bibles")
Religion = grouped.get_group("Religion & Spirituality")
Sports = grouped.get_group("Sports & Outdoors")
Teen = grouped.get_group("Teen & Young Adult")
Childrens = grouped.get_group("Children's Books")
Travel = grouped.get_group("Travel")

GenreGroups = [Calendars['titles'], Comics['titles'], Test['titles'], Mystery['titles'], SciFi['titles'], 
               Romance['titles'], Humor['titles'], Literature['titles'], LGBTQ['titles'], Engineering['titles'], 
               Food['titles'], Crafts['titles'], Arts['titles'], Education['titles'], Parenting['titles'], 
               SelfHelp['titles'], Computers['titles'], Medical['titles'], Science['titles'], Health['titles'], 
               Business['titles'], Law['titles'], Biographies['titles'], History['titles'], Politics['titles'], 
               Reference['titles'], Bibles['titles'], Religion['titles'], Sports['titles'], Teen['titles'], 
               Childrens['titles'], Travel['titles']]

In [26]:
# token frequencies in each genre
genre_count = []
for g in GenreGroups:
    genre_title = Counter()
    for title in g:
        for i in title:
            genre_title[i] += 1
    genre_count.append(genre_title)

# combine genres descriptions and titles grouped by genres
zipped_values = zip(df1.genre.unique(), genre_count)
tok_freq_genres = list(zipped_values)

In [30]:
# token frequencies in each genre
tok_freq_genres[0][1] # first index specifies genre, second index specifies token frequencies vs genre label

Counter({'doug': 1,
         'the': 382,
         'pug': 2,
         '2016': 1130,
         'wall': 805,
         'calendar': 1970,
         'sierra': 3,
         'club': 5,
         'engagement': 64,
         'wilderness': 11,
         'thomas': 12,
         'kinkade': 9,
         'disney': 22,
         'dreams': 5,
         'collection': 11,
         'ansel': 3,
         'adams': 3,
         'dilbert': 5,
         'daytoday': 92,
         'mary': 7,
         'engelbreit': 5,
         'deluxe': 33,
         'never': 3,
         'give': 1,
         'up': 9,
         'cat': 28,
         'pageaday': 56,
         'gallery': 12,
         'llewellyns': 6,
         'witches': 3,
         'datebook': 3,
         'amy': 3,
         'knapp': 2,
         'big': 12,
         'grid': 4,
         'essential': 1,
         'organization': 1,
         'and': 147,
         'communication': 1,
         'tool': 1,
         'for': 65,
         'entire': 1,
         'family': 10,
         'outlander': 2,
 

In [31]:
# number of words in a class
len(tok_freq_genres[0][1]) # first genre

2379

In [33]:
# number of total vocabulary (training set)
V = len(Counter(tok_freq))
print(V)

74628


In [34]:
# number of titles (in training set)
N_titles = len(train)
print(N_titles)

165931


In [35]:
# number of titles in each genre
N_genre = train['genres'].value_counts()
print(N_genre[:5])

Travel                       14636
Children's Books             10841
Medical Books                 9675
Health, Fitness & Dieting     9493
Crafts, Hobbies & Home        7980
Name: genres, dtype: int64


In [93]:
# priors of each genre (probability of title being in specific genre)
prob_title = []
for g in range(len(N_genre)):
    prob = N_genre[g] / N_titles
    prob_title.append(prob)

zipped_values = zip(df1.genre.unique(), prob_title)
prior = list(zipped_values)

print(prior[:5]) # test

[('Calendars', 0.08820533836353665), ('Comics & Graphic Novels', 0.06533438597971446), ('Test Preparation', 0.0583073687255546), ('Mystery, Thriller & Suspense', 0.05721052726735812), ('Science Fiction & Fantasy', 0.048092279320922554)]


In [123]:
# likelihoods of each word (probability of word being in specific genre)

# number of times word appears in genre = token frequencies in genres = tok_freq_genres[i][1]["word"]
# number of words in genre = len(tok_freq_genres[i][1])
# total vocab = V
# list of all possible words in vocab = dict.keys(tok_freq)

# create empty dataframe
likelihood = pd.DataFrame(columns=df1.genre.unique(), index=dict.keys(tok_freq))

for i in tok_freq_genres: # loop through genres
    for word in i[1]: # loop through words in genre -> returns list of all words in one genre instead of just one word at a time
      for w in word: # this just gives list of separate letters ??
        p = (tok_freq_genres[i][1][word] + 1) / (len(tok_freq_genres[i][1]) + V) # this line technically works -> word variable doesn't
        likelihood.at[word, i] = p # p value should replace NaN at specified index -> doesn't work

# how to fill probabilities of words that haven't appeared in genre? -> using if statement for all NaN values in dataframe
# if word not in tok_freq_genres
# p = (0+1)/(len(tok_freq_genres[i][1]) + V)

TypeError: ignored

In [132]:
genre_count

[Counter({'doug': 1,
          'the': 382,
          'pug': 2,
          '2016': 1130,
          'wall': 805,
          'calendar': 1970,
          'sierra': 3,
          'club': 5,
          'engagement': 64,
          'wilderness': 11,
          'thomas': 12,
          'kinkade': 9,
          'disney': 22,
          'dreams': 5,
          'collection': 11,
          'ansel': 3,
          'adams': 3,
          'dilbert': 5,
          'daytoday': 92,
          'mary': 7,
          'engelbreit': 5,
          'deluxe': 33,
          'never': 3,
          'give': 1,
          'up': 9,
          'cat': 28,
          'pageaday': 56,
          'gallery': 12,
          'llewellyns': 6,
          'witches': 3,
          'datebook': 3,
          'amy': 3,
          'knapp': 2,
          'big': 12,
          'grid': 4,
          'essential': 1,
          'organization': 1,
          'and': 147,
          'communication': 1,
          'tool': 1,
          'for': 65,
          'entire': 1,
       

In [155]:
# probability of word "dog" being in genre "Calendars"
p = (tok_freq_genres[0][1]["dog"] + 1) / (len(tok_freq_genres[0][1]) + V)
print(p)

0.00025971664913579287


In [157]:
# check whether value was actually replaced at specified index
likelihood.loc[["dog"]]

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel,0
dog,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00026


In [152]:
# empty dataframe
likelihood = pd.DataFrame(columns=df1.genre.unique(), index=dict.keys(tok_freq))
print(likelihood)

              Calendars Comics & Graphic Novels  ... Children's Books Travel
doug                NaN                     NaN  ...              NaN    NaN
the                 NaN                     NaN  ...              NaN    NaN
pug                 NaN                     NaN  ...              NaN    NaN
2016                NaN                     NaN  ...              NaN    NaN
wall                NaN                     NaN  ...              NaN    NaN
...                 ...                     ...  ...              ...    ...
ballrooms           NaN                     NaN  ...              NaN    NaN
riverboat           NaN                     NaN  ...              NaN    NaN
historymapped       NaN                     NaN  ...              NaN    NaN
1790                NaN                     NaN  ...              NaN    NaN
fairfax             NaN                     NaN  ...              NaN    NaN

[74628 rows x 32 columns]


In [89]:
# number of times "the" appears in first genre
tok_freq_genres[0][1]["the"]

382

In [ ]:
# test NB using test data

# predictions on word by word basis using words in titles of test data
# for each word in test title do: prior * likelihood^2

## BERT classifier

In [ ]:
# import relevant packages
import transformers

# do we need PyTorch / Tensorflow as well?